# deberta-v3-small Experiments

|     Model              |  CFG | cv score | lb score | comment|
|:----------------------|:--------|:--------:|:--------:|:--------|
|  Baseline                        | `full_baseline_cfg`      |   |  | |
|  Context Text                    | `full_ctxt_cfg`          |   |  | |  
|  Chemical Lookup                 | `full_post_chem_cfg`     |   |  | |
|  Neighbors Augmentation          | `full_aug_neighbors_cfg` |   |  | |     
|  Chemical Compounds Augmentation | `fast_aug_chem_cfg`      |   |  | | 


In [14]:
# ====================================================
# Directory settings
# ====================================================
import os

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Model Configuration

In [15]:
# ====================================================
# CFG
# ====================================================
from dataclasses import dataclass, field
from typing import Set, Optional, Tuple, Dict


@dataclass
class ModelConfig:
    chem_comp_pred_avg: bool  = False
    prompt_engineering: Optional[str] = 'ctx_txt' # None
    
        
    num_workers=4
    path="../input/uspppm-deberta-v3-small-experiments-train/"
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-small"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=116
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    map_labels = {0: 0.0, 1: 0.25, 2: 0.5, 3: 0.75, 4: 1.0}
    
# Configurations
full_baseline_cfg = ModelConfig(prompt_engineering=None)
full_ctxt_cfg = ModelConfig()
full_post_chem_cfg = ModelConfig(chem_comp_pred_avg=True)
full_aug_neighbors_cfg = ModelConfig() # TODO!
full_aug_chem_cfg = ModelConfig() # TODO!

### Config used throughout this notebook

In [16]:
#CFG = full_baseline_cfg
# CFG = full_ctxt_cfg
CFG = full_post_chem_cfg
# CFG = full_aug_neighbors_cfg
# CFG = fast_aug_chem_cfg

# Library

In [17]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import shutil
import string
import pickle
import random
import joblib
import itertools
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
from scipy import stats
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels-dataset tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# common chemical formulae lookup utility script
from uspppm_common_chemical_compound_lookup import USPPPMChemCompLookup

torch.__version__: 1.9.1
Found existing installation: transformers 4.16.2
Uninstalling transformers-4.16.2:
  Successfully uninstalled transformers-4.16.2


Found existing installation: tokenizers 0.11.6
Uninstalling tokenizers-0.11.6:
  Successfully uninstalled tokenizers-0.11.6


Looking in links: ../input/pppm-pip-wheels-dataset
Processing /kaggle/input/pppm-pip-wheels-dataset/transformers-4.16.2-py3-none-any.whl
Processing /kaggle/input/pppm-pip-wheels-dataset/tokenizers-0.11.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


Looking in links: ../input/pppm-pip-wheels-dataset
tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


# Utils

### Chemical Compound Lookup

In [18]:
chem_lookup = USPPPMChemCompLookup(chem_comp_path='../input/chemical-compounds-lookup/compounds.csv')

**Chemical Compound Lookup Tests**

In [19]:
print('Testing basic formula lookup...')
print(chem_lookup.lookup_df)
print(chem_lookup.phrase_chem_formula_synonym('agbr test'))
print(chem_lookup.phrase_chem_formula_synonym('agbr dna test agonc ag2cl2'))
print(chem_lookup.phrase_chem_formula_synonym('dna test d2o'))
print(chem_lookup.chem_formula_synonyms('c3h6'))
print('Done!')
print()
print('Testing train dataset augmentation...')
chem_test = pd.DataFrame({'id': pd.Series(['t1', 't2', 't3', 't4']),
                          'score': pd.Series([3, 2, 1, 0]),
                          'anchor': pd.Series(['agbr dna test agonc ag2cl2', 'agbr test', 'agbr', 'last']),
                          'target': pd.Series(['agonc ag2cl2', 'test thingy', 'c4h7no4', 'last']),
                          'context': pd.Series(['G02', 'G02', 'C12', 'C12'])})
print("Before")
print(chem_test)
print("After")
print(chem_lookup.pre_augment_chem_formulae(chem_test, True))
print('Done!')
print()
print('Testing test dataset augmentation...')
chem_test = pd.DataFrame({'id': pd.Series(['t1', 't2', 't3', 't4']),
                          'text': pd.Series([
                              'agbr dna test agonc ag2cl2 [SEP] agonc ag2cl2 [SEP] G02',
                              'agbr test [SEP] test thingy [SEP] G02',
                              'agbr [SEP] c4h7no4 [SEP] C12',
                              'last [SEP] last [SEP] C12'
                          ])})
print("Before")
print(chem_test)
print("After")
print(chem_lookup.post_augment_chem_formulae(chem_test, True))
print('Done!')

Testing basic formula lookup...
                            Name  Formula
0               actiniumiiioxide    ac2o3
1     silvertetrachloroaluminate  agalcl4
2                  silverbromide     agbr
3                  silverbromate   agbro3
4                  silvercyanide     agcn
...                          ...      ...
4067                zirconateion    zro32
4068          zirconiumphosphide     zrp2
4069            zirconiumsulfide     zrs2
4070           zirconiumsilicide    zrsi2
4071          zirconiumphosphate  zr3po44

[4072 rows x 2 columns]
[('silverbromide test', True)]
[('silverbromide dna test silverfulminate disilverdichloride', True), ('silverbromide dna test silvercyanate disilverdichloride', True), ('silverbromide dna test silverfulminate silveriidichloride', True), ('silverbromide dna test silvercyanate silveriidichloride', True)]
[('dna test deuteriumoxide', True), ('dna test heavywater', True)]
['cyclopropane', 'propylene']
Done!

Testing train dataset augmentat

In [20]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

# OOF

In [21]:
oof_df = pd.read_csv(CFG.path+'oof_df.csv')
labels = oof_df['score'].values
preds = oof_df['pred'].values
score = get_score(labels, preds)
LOGGER.info(f'CV Score: {score:<.4f}')

CV Score: 0.8362
CV Score: 0.8362


# Data Loading

In [22]:
# ====================================================
# Data Loading
# ====================================================
test = pd.read_csv(INPUT_DIR+'test.csv')
submission = pd.read_csv(INPUT_DIR+'sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
# display(test.head())
# display(submission.head())

test.shape: (36, 4)
submission.shape: (36, 2)


In [23]:
# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load(CFG.path+"cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)
# display(test.head())

In [24]:
def get_sec_toks(df):
    return '[' + df['context'].str[0] + ']'

if CFG.prompt_engineering == 'custom_tok':
    test['text'] = get_sec_toks(test) + test['context_text'] + '[SEP]' + test['anchor'] + CFG.custom_sep_token + test['target']
elif CFG.prompt_engineering == 'ctx_txt':
    test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context_text']
elif CFG.prompt_engineering == None:
    test['text'] = test['anchor'] + '[SEP]' + test['target'] + '[SEP]'  + test['context']
else:
    raise(ValueError('CFG.prompt_engineering = {} not recognized!'.format(CFG.prompt_engineering)))
    
display(test['text'].head())

0    opc drum[SEP]inorganic photoconductor drum[SEP...
1    adjust gas flow[SEP]altering gas flow[SEP]MECH...
2    lower trunnion[SEP]lower locating[SEP]PERFORMI...
3    cap component[SEP]upper portion[SEP]TEXTILES; ...
4    neural stimulation[SEP]artificial neural netwo...
Name: text, dtype: object

# tokenizer

In [25]:
# ====================================================
# tokenizer
# ====================================================
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.path+'tokenizer/')
print(CFG.tokenizer.all_special_tokens)

['[CLS]', '[SEP]', '[UNK]', '[PAD]', '[MASK]']


# Dataset

In [26]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df, chem_lookup=None):
        self.cfg = cfg
        self.texts = df['text'].values
        self.ids = df['id'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs, self.ids[item]

# Model

In [27]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

# inference

In [28]:
def average_by_id(df):
    ''' Averages a dataframe by a column id'''
    orig_id_order = df['id']
    unordered_means = df.groupby('id').mean().reset_index()
    return unordered_means.set_index('id').loc[orig_id_order].reset_index().drop_duplicates()

# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    ids = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs, sample_id in tk0:
        ids.append(sample_id)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    ids = np.concatenate(ids)
    if CFG.chem_comp_pred_avg:
        pred_new = average_by_id(pd.DataFrame({'pred': np.hstack(predictions), 'id': ids}))
        predictions = pred_new['pred'].to_numpy()
    return predictions

In [29]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(CFG.path+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
# predictions, _ = stats.mode(predictions, axis=0)
predictions = np.mean(predictions, axis=0)

  0%|          | 0/2 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

  0%|          | 0/2 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

# Submission

In [30]:
submission['score'] = predictions
# submission['score'] = predictions[0]
# display(submission.head())
submission[['id', 'score']].to_csv('submission.csv', index=False)